In [15]:
# Setup dell'ambiente
# In questa cella installiamo e importiamo le librerie necessarie.
# Utilizziamo yfinance per il download dei dati di mercato,
# pandas e numpy per la manipolazione dei dati,
# matplotlib per la visualizzazione.

!pip -q install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# Download dei prezzi
# In questa funzione scarichiamo i prezzi giornalieri di un asset finanziario.
# Utilizziamo l'Adj Close quando disponibile, per evitare distorsioni
# dovute a split o dividendi.

def download_prices(ticker, start, end):
    df = yf.download(
        ticker,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False
    )

    if df.empty:
        raise ValueError(f"Nessun dato scaricato per {ticker}.")

    # Normalizziamo i nomi delle colonne
    df = df.rename(columns=str.title)

    # Se disponibile, utilizziamo Adj Close
    if "Adj Close" in df.columns:
        df = df[["Adj Close"]].rename(columns={"Adj Close": "price"})
    else:
        df = df[["Close"]].rename(columns={"Close": "price"})

    df.index = pd.to_datetime(df.index)
    return df

In [17]:
# Calcolo dei rendimenti logaritmici e volatilità
# In questa funzione calcoliamo:
# 1) i rendimenti logaritmici giornalieri
# 2) la volatilità storica come deviazione standard mobile
#    dei rendimenti, su finestre temporali definite.

def compute_volatility(df, windows=(5, 10)):
    out = df.copy()

    # Rendimenti logaritmici
    out["log_return"] = np.log(out["price"] / out["price"].shift(1))

    # Volatilità rolling
    for w in windows:
        out[f"vol_{w}d"] = out["log_return"].rolling(window=w).std()

    return out

In [18]:
#Estrazione finestra evento
# In questa funzione estraiamo una finestra evento specifica.
# Inseriamo un buffer temporale precedente all'evento per
# garantire il calcolo corretto della volatilità rolling.

def event_window(df, event_start, event_end, vol_window=5):
    event_start = pd.to_datetime(event_start)
    event_end = pd.to_datetime(event_end)

    # Buffer precedente all'evento
    buffer_start = event_start - pd.tseries.offsets.BDay(vol_window * 2)

    tmp = df.loc[
        (df.index >= buffer_start) &
        (df.index <= event_end)
    ].copy()

    # Ritagliamo la finestra evento finale
    win = tmp.loc[
        (tmp.index >= event_start) &
        (tmp.index <= event_end)
    ].copy()

    return win

In [19]:
#Definizione delle finestre temporali di crisi
# In questa cella definiamo le finestre temporali di interesse,
# coerenti con gli eventi di mercato analizzati nel progetto.

# Crisi SVB (proxy: KRE ETF)
svb_start = "2023-03-10"
svb_end   = "2023-03-17"

# Evento Netflix (earnings shock)
nflx_start = "2022-04-19"
nflx_end   = "2022-05-03"

# Finestre di volatilità rolling
VOL_WINDOWS = (5, 10)

In [20]:
#Analisi volatilità KRE
# In questa cella analizziamo la volatilità del settore bancario
# utilizzando l'ETF KRE come proxy della crisi SVB.

# Scarichiamo i dati includendo un periodo precedente all'evento
kre_raw = download_prices(
    ticker="KRE",
    start="2023-02-01",
    end="2023-03-31"
)

# Calcoliamo rendimenti e volatilità
kre = compute_volatility(kre_raw, windows=VOL_WINDOWS)

# Estraiamo la finestra evento
kre_svb = event_window(
    kre,
    event_start=svb_start,
    event_end=svb_end,
    vol_window=min(VOL_WINDOWS)
)

kre_svb

Price,price,log_return,vol_5d,vol_10d
Ticker,Kre,,,
Date,,,,
2023-03-10,46.320663,-0.044941,0.032439,0.028534
2023-03-13,40.618542,-0.131364,0.049173,0.044378
2023-03-14,41.468380,0.020707,0.060921,0.046622
2023-03-15,40.792160,-0.016441,0.059074,0.046128
2023-03-16,42.226833,0.034566,0.065936,0.050687
2023-03-17,39.695599,-0.061816,0.067466,0.049954


In [21]:
#Analisi volatilità Netflix
# In questa cella analizziamo la volatilità del titolo Netflix
# in corrispondenza dello shock informativo di aprile–maggio 2022.

# Scarichiamo i dati includendo un periodo precedente all'evento
nflx_raw = download_prices(
    ticker="NFLX",
    start="2022-03-01",
    end="2022-05-20"
)

# Calcoliamo rendimenti e volatilità
nflx = compute_volatility(nflx_raw, windows=VOL_WINDOWS)

# Estraiamo la finestra evento
nflx_evt = event_window(
    nflx,
    event_start=nflx_start,
    event_end=nflx_end,
    vol_window=min(VOL_WINDOWS)
)

nflx_evt

Price,price,log_return,vol_5d,vol_10d
Ticker,Nflx,,,
Date,,,,
2022-04-19,34.861000,0.031322,0.023749,0.020722
2022-04-20,22.618999,-0.432578,0.196245,0.135201
2022-04-21,21.822001,-0.035872,0.190616,0.135135
2022-04-22,21.552000,-0.012450,0.192006,0.135274
2022-04-25,20.990999,-0.026375,0.190353,0.135051
2022-04-26,19.840000,-0.056394,0.179511,0.134628
2022-04-27,18.854000,-0.050975,0.017934,0.133739
2022-04-28,19.952000,0.056604,0.045355,0.136755


In [11]:
# In questa cella controlliamo le colonne finali disponibili,
# pronte per il merge con i segnali informativi
# (sentiment, volume news, model disagreement index).

print("KRE – finestra SVB")
display(kre_svb.head())

print("\nNetflix – finestra evento")
display(nflx_evt.head())


KRE – finestra SVB


Price,price,log_return,vol_5d,vol_10d
Ticker,Kre,,,
Date,,,,
2023-03-10,46.320663,-0.044941,0.032439,0.028534
2023-03-13,40.618542,-0.131364,0.049173,0.044378
2023-03-14,41.468380,0.020707,0.060921,0.046622
2023-03-15,40.792160,-0.016441,0.059074,0.046128
2023-03-16,42.226833,0.034566,0.065936,0.050687



Netflix – finestra evento


Price,price,log_return,vol_5d,vol_10d
Ticker,Nflx,,,
Date,,,,
2022-04-19,34.861000,0.031322,0.023749,0.020722
2022-04-20,22.618999,-0.432578,0.196245,0.135201
2022-04-21,21.822001,-0.035872,0.190616,0.135135
2022-04-22,21.552000,-0.012450,0.192006,0.135274
2022-04-25,20.990999,-0.026375,0.190353,0.135051


In [12]:
# In questa cella rendiamo esplicita la colonna 'date'
# (necessaria per i merge successivi).

kre_svb_save = kre_svb.reset_index().rename(columns={"Date": "date"})
nflx_evt_save = nflx_evt.reset_index().rename(columns={"Date": "date"})

kre_svb_save["date"] = pd.to_datetime(kre_svb_save["date"])
nflx_evt_save["date"] = pd.to_datetime(nflx_evt_save["date"])


In [13]:
# In questa cella salviamo i dataframe come file CSV,
# che useremo successivamente per il merge con i segnali informativi.

kre_svb_save.to_csv("kre_svb_event_window.csv", index=False)
nflx_evt_save.to_csv("nflx_event_window.csv", index=False)

print("File salvati correttamente:")
print("- kre_svb_event_window.csv")
print("- nflx_event_window.csv")


File salvati correttamente:
- kre_svb_event_window.csv
- nflx_event_window.csv


In [22]:
# Funzione per riempire le date mancanti con NA
def fill_missing_dates_with_na(df, start_date, end_date):
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    # Convert index to DatetimeIndex to ensure proper reindexing
    df.index = pd.to_datetime(df.index)
    # Reindex the DataFrame
    df_reindexed = df.reindex(full_date_range)
    df_reindexed.index.name = 'Date'
    return df_reindexed

# Applichiamo la funzione a kre_svb
kre_svb_reindexed = fill_missing_dates_with_na(kre_svb, svb_start, svb_end)

# Applichiamo la funzione a nflx_evt
nflx_evt_reindexed = fill_missing_dates_with_na(nflx_evt, nflx_start, nflx_end)

print("KRE – finestra SVB (con NA per giorni mancanti)")
display(kre_svb_reindexed)

print("\nNetflix – finestra evento (con NA per giorni mancanti)")
display(nflx_evt_reindexed)

KRE – finestra SVB (con NA per giorni mancanti)


Price,price,log_return,vol_5d,vol_10d
Ticker,Kre,,,
Date,,,,
2023-03-10,46.320663,-0.044941,0.032439,0.028534
2023-03-11,NaN,NaN,NaN,NaN
2023-03-12,NaN,NaN,NaN,NaN
2023-03-13,40.618542,-0.131364,0.049173,0.044378
2023-03-14,41.468380,0.020707,0.060921,0.046622
2023-03-15,40.792160,-0.016441,0.059074,0.046128
2023-03-16,42.226833,0.034566,0.065936,0.050687
2023-03-17,39.695599,-0.061816,0.067466,0.049954



Netflix – finestra evento (con NA per giorni mancanti)


Price,price,log_return,vol_5d,vol_10d
Ticker,Nflx,,,
Date,,,,
2022-04-19,34.861000,0.031322,0.023749,0.020722
2022-04-20,22.618999,-0.432578,0.196245,0.135201
2022-04-21,21.822001,-0.035872,0.190616,0.135135
2022-04-22,21.552000,-0.012450,0.192006,0.135274
2022-04-23,NaN,NaN,NaN,NaN
2022-04-24,NaN,NaN,NaN,NaN
2022-04-25,20.990999,-0.026375,0.190353,0.135051
2022-04-26,19.840000,-0.056394,0.179511,0.134628


In [29]:
def flatten_columns_keep_all(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.index.name = "Date"

    if isinstance(df.columns, pd.MultiIndex):
        # concatena TUTTI i livelli: es. ("Price","NFLX") > "Price_NFLX"
        df.columns = [
            "_".join([str(x) for x in col if str(x).lower() not in ("nan", "none", "")]).strip("_")
            for col in df.columns.to_list()
        ]
    else:
        df.columns = [str(c) for c in df.columns]

    df.columns.name = None
    return df

def add_ticker_column(df: pd.DataFrame, ticker_label: str) -> pd.DataFrame:
    df = df.copy()
    df.insert(0, "Ticker", ticker_label)
    return df

# Netflix
nflx_out = add_ticker_column(flatten_columns_keep_all(nflx_evt_reindexed), "Netflix")
nflx_out.to_csv("Netflix_Event_Table_with_NaN.csv", index=True, na_rep="NaN")

# SVB (KRE)
svb_out = add_ticker_column(flatten_columns_keep_all(kre_svb_reindexed), "SVB")
svb_out.to_csv("SVB_KRE_Event_Table_with_NaN.csv", index=True, na_rep="NaN")

print("OK: CSV esportati con Ticker + tutte le colonne originali.")
print("- Netflix_Event_Table_with_NaN.csv")
print("- SVB_KRE_Event_Table_with_NaN.csv")

OK: CSV esportati con Ticker + tutte le colonne originali.
- Netflix_Event_Table_with_NaN.csv
- SVB_KRE_Event_Table_with_NaN.csv
